# Chain‑of‑Thought Faithfulness Experiments

In [1]:
import os, pathlib, sys
%cd ..
print("cwd ->", os.getcwd())
print("file exists? ->", pathlib.Path('g_cot_cluster/outputs/mmlu/DeepSeek-R1-Distill-Llama-8B/segmented_completions_none.json').exists())

/root/CoTFaithChecker
cwd -> /root/CoTFaithChecker
file exists? -> True


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


Install dependencies if needed (`pip install transformers scikit-learn umap-learn cpca matplotlib`) and import helper modules.

In [2]:
# needs:
"""
pip install umap
pip install contrastive
"""

import os, sys, json, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt
%matplotlib inline

# add project path
sys.path.append("/mnt/data/cot_faithfulness")

from g_cot_cluster.cot_direct.data_loader import load_segmented_completions, iter_segments
from g_cot_cluster.cot_direct.representations import RepresentationExtractor
from g_cot_cluster.cot_direct.clustering import cluster_kmeans, embed_to_umap, plot_clusters_2d
from g_cot_cluster.cot_direct.cpca_utils import run_cpca, plot_cpca_projection
from g_cot_cluster.cot_direct.probes import layerwise_probe
from g_cot_cluster.cot_direct.causal_edit import ActivationPatcher


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
DATA_DIR = Path("g_cot_cluster/outputs/mmlu/DeepSeek-R1-Distill-Llama-8B")
HINT_TYPES = ["none", "sycophancy", "induced_urgency", "unethical_information"]
MODEL_NAME = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"

extractor = RepresentationExtractor(MODEL_NAME)

NameError: name 'tokenizer' is not defined

## 2‑a. Clustering representations by category

In [ ]:
import pandas as pd
results = []
for hint in HINT_TYPES:
    file_path = DATA_DIR / f"segmented_completions_{hint}.json"
    data = load_segmented_completions(file_path)
    segs = list(iter_segments(data))
    texts = [s["text"] for s in segs]
    cats  = [s["phrase_category"] for s in segs]
    emb   = extractor.bulk_embed(texts, layer=-1)
    labels, sil = cluster_kmeans(emb, n_clusters=len(set(cats)))
    um2 = embed_to_umap(emb)
    fig = plot_clusters_2d(um2, [list(sorted(set(cats))).index(c) for c in cats],
                           f"UMAP projection by category – {hint}")
    plt.show()
    results.append({"hint": hint, "silhouette": sil})
pd.DataFrame(results).set_index("hint").plot(kind="bar", legend=False, title="Silhouette scores")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '.cache/embeddings/9079943670916444799_L-1.npy'

## 2‑b. Contrastive PCA

In [ ]:

bg_emb = extractor.bulk_embed(
    [s["text"] for s in iter_segments(load_segmented_completions(DATA_DIR / "segmented_completions_none.json"))]
)
for hint in HINT_TYPES[1:]:
    tgt_emb = extractor.bulk_embed(
        [s["text"] for s in iter_segments(load_segmented_completions(DATA_DIR / f"segmented_completions_{hint}.json"))]
    )
    model = run_cpca(background=bg_emb, target=tgt_emb)
    fig = plot_cpca_projection(model, bg_emb, tgt_emb, f"cPCA – {hint} vs none")
    plt.show()


## 2‑c. Probing predictability of categories

In [ ]:

data_none = load_segmented_completions(DATA_DIR / "segmented_completions_none.json")
segs = list(iter_segments(data_none))
texts = [s["text"] for s in segs]
cats = [s["phrase_category"] for s in segs]
label_map = {c:i for i,c in enumerate(sorted(set(cats)))}
y = np.array([label_map[c] for c in cats])

# Example: only last layer; extend to all layers as needed
reps_last = extractor.bulk_embed(texts, layer=-1)
layer_reps = reps_last[:, None, :]  # shape (n, 1, d)
accs = layerwise_probe(layer_reps, y)
print("Probe accuracy, last layer:", accs[0])


## 2‑d. Activation patching demo

In [ ]:

patcher = ActivationPatcher(MODEL_NAME)
sample_src = segs[0]["text"]
sample_donor = segs[1]["text"]
logits = patcher.patch(sample_src, sample_donor, num_tokens=20, layer_idx=-1)
print("Patched logits shape:", logits.shape)
